In [ ]:
import earthaccess
import numpy as np
import xarray as xr
from cartopy.crs import Orthographic, PlateCarree
from matplotlib import pyplot
from PIL import Image, ImageEnhance

In [ ]:
tspan = ("2024-08", "2024-08")
results = earthaccess.search_data(
    short_name="PACE_OCI_L3M_SFREFL",
    granule_name="*.MO.*.0p1deg.*",
    temporal=tspan,
)
paths = earthaccess.download(results, "granules")

In [ ]:
dataset = xr.open_dataset(paths[0])

In [ ]:
dataset

In [ ]:
rgb = dataset["rhos"].sel({"wavelength": [645, 555, 368]}, method="nearest")

In [ ]:
plot = rgb.plot.imshow()

In [ ]:
scale = 0.01
vmin = 0.01
vmax = 1.02
gamma = 0.95
contrast = 1.5
brightness = 1.02
sharpness = 2
saturation = 1.1

rgb = rgb.where(rgb > 0)
rgb = np.log(rgb / scale) / np.log(1 / scale)
rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min())
rgb = rgb * gamma
img = rgb * 255
img = img.where(img.notnull(), 0).astype("uint8")
img = Image.fromarray(img.data)
enhancer = ImageEnhance.Contrast(img)
img = enhancer.enhance(contrast)
enhancer = ImageEnhance.Brightness(img)
img = enhancer.enhance(brightness)
enhancer = ImageEnhance.Sharpness(img)
img = enhancer.enhance(sharpness)
enhancer = ImageEnhance.Color(img)
img = enhancer.enhance(saturation)
rgb[:] = np.array(img) / 255

In [ ]:
plot = rgb.plot.imshow()

In [ ]:
fig, ax = pyplot.subplots(figsize=(13, 4), subplot_kw={"projection": Orthographic(-30)})
rgb.plot.imshow(ax=ax, transform=PlateCarree())
pyplot.show()

In [ ]:
pyplot.savefig(f"{tspan[0]}-blue-marble.png")